In [ ]:
import requests
from os import getenv
from dotenv import load_dotenv
from json import loads, dumps
from IPython.display import display, Markdown
from pprint import pprint

In [ ]:
load_dotenv(override = True)

GITHUB_ACCESS_TOKEN = getenv('GITHUB_ACCESS_TOKEN')

if not GITHUB_ACCESS_TOKEN:
    print('No Token found')

In [ ]:
from pydantic import BaseModel, Field, TypeAdapter

class Change(BaseModel):
    file_name: str
    status: str = Field(description = "Added/Modified/Deleted")
    patch: str = Field(description = "Changes made")

class Progress(BaseModel):
    task_summary: str = Field(description = "Title of task")
    recent_activity: list[str] = Field(description = "List of commit messages")
    changes: list[Change]

In [ ]:
BASE_URI = "https://api.github.com"
COMPARE_PATH = "/repos/{owner}/{repository}/compare/{base_branch}...{feature_branch}"

In [ ]:
owner = 'KarthikReddyMaru'
repository = 'test'
base_branch = 'main'
feature_branch = 'feat/ryomen'

URI = BASE_URI + COMPARE_PATH.format(
    owner = owner, 
    repository = repository, 
    base_branch = base_branch,
    feature_branch = feature_branch
)

headers = {
    'Authorization': f"Bearer {GITHUB_ACCESS_TOKEN}"
}

response = requests.get(url = URI, headers = headers)


In [ ]:
content = loads(response.content)

In [ ]:
task_summary = f"User is working on {feature_branch}"
recent_activity = [commit['commit']['message'] for commit in content['commits']]

changes = []

for change in content['files']:
    changes.append(
        Change(
            file_name = change['filename'],
            status = change['status'],
            patch = change['patch']
        )
    )

progress = Progress(task_summary = task_summary, recent_activity = recent_activity, changes = changes)

recent_activity_json = dumps(recent_activity)

changesTypeAdapter = TypeAdapter(type = list[Change])
changes_json = changesTypeAdapter.dump_python(progress.changes)

In [ ]:
from langchain_ollama import ChatOllama
from langchain.messages import HumanMessage, SystemMessage
from langchain_core.messages import convert_to_messages

OLLAMA_MODEL = 'gpt-oss:latest'
LLM = ChatOllama(model = OLLAMA_MODEL)

In [ ]:
SYSTEM_PROMPT = """
    You are the developer working on a specific task. You are chatting with your Project Manager.

    **CRITICAL RULE: CONVERSATIONAL CONTEXT**
    - If the user says "Hi", "Hello", or "How are you?", **YOU MUST ONLY REPLY WITH A GREETING.** Do NOT mention code, files, or status. Just say "Hey, what's up?" or "Hi there."
    - **ONLY** report status if the user explicitly asks questions like "What's the status?", "What did you do?", "Is the login page done?", or "Any blockers?".

    **Your Persona:**
    - You are a normal human developer, not a robot.
    - You are casual but professional.
    - You use "I" because this is your code.

    **Answering Work Questions:**
    - Use the provided {changes} to answer truthfully.
    - Explain things in simple English (Business Value), not code syntax.
    - If the user asks about something NOT in your specific task/files, say: "I'm not working on that part right now," or "I don't see that in my current changes."

    **Example 1 (Greeting):**
    User: "Hi"
    You: "Hey! What can I help you with?" (CORRECT)
    You: "Hi, I just updated the API." (WRONG - Don't volunteer info unprompted)

    **Example 2 (Status):**
    User: "How is the task going?"
    You: "Going well. I just finished the movie list feature and added a few songs to the playlist."
"""


In [ ]:
USER_PROMPT = """
    ### MY CURRENT WORK (For reference ONLY if asked)
    Task: {task_summary}
    Changes: {changes}

    ### CONVERSATION
    Manager: {message}
    Developer (You):
"""


In [ ]:
def chat(message, history):
    
    user = USER_PROMPT.format(
        task_summary = task_summary,
        recent_activity = recent_activity_json,
        changes = changes_json,
        message = message
    )
    
    # 2. Convert past history to LangChain format
    history_messages = convert_to_messages(history)
    
    # 3. Construct the full message list: System -> History -> Current User
    full_messages = [SystemMessage(content = SYSTEM_PROMPT)] + history_messages + [HumanMessage(content = user)]

    stream = LLM.stream(input = full_messages)

    response = ''
    for chunk in stream:
        response += chunk.content or ''
        yield response


In [ ]:
import gradio as gr 

gr.ChatInterface(fn = chat).launch()